In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    ################################################ s', r = env(s, a)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    next_states_logits, Qlogits = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                      action_size=action_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss_ = tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, labels=next_states_labels)
    eloss = tf.reduce_mean(tf.reduce_sum(eloss_, axis=1))
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, nextQlogits = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                          action_size=action_size, state_size=state_size, reuse=True)
    #################################################### GAN
    Qlogits = tf.reshape(Qlogits, shape=[-1])
    nextQlogits = tf.reshape(nextQlogits, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(nextQlogits) # minimize nextQ
    aloss = -tf.reduce_mean(nextQlogits) # maximize nextQ
    ###################################################### Qlearning
    ###################################################### Q(s,a)= r + Q'(s',a')
    rewards = tf.reduce_mean(tf.square(Qlogits - (gamma * nextQlogits)))
    ##################################################### GAN + DDPG
    eloss += tf.reduce_mean(rewards) # minimize entropy=error
    aloss += -tf.reduce_mean(rewards) # maximize entropy=rewards
    return actions_logits, aloss, eloss, rewards

In [7]:
def model_opt(a_loss, e_loss, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
    return a_opt, e_opt

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.rewards_ = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt = model_opt(a_loss=self.a_loss, e_loss=self.e_loss,
                                           a_learning_rate=a_learning_rate,
                                           e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        #print(done)
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [14]:
def minibacth(memory):
    # Training with the maxrated minibatch
    batch = memory.buffer
    #for idx in range(memory_size// batch_size):
    while True:
        idx = np.random.choice(np.arange(memory_size// batch_size))
        states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        states = states[rates >= np.max(rates)]
        actions = actions[rates >= np.max(rates)]
        next_states = next_states[rates >= np.max(rates)]
        rewards = rewards[rates >= np.max(rates)]
        dones = dones[rates >= np.max(rates)]
        rates = rates[rates >= np.max(rates)]
        if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
            break
    return states, actions, next_states, rewards, dones, rates

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        rate = -1

        # Training steps/batches
        for num_step in range(11111):
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            total_reward += reward
            state = next_state
            
            # Online policy/on-policy/online training
            states, actions, next_states, rewards, dones, rates = minibacth(memory=memory)
            feed_dict = {model.states: states, model.actions: actions, model.next_states: next_states,
                         model.rewards: rewards, model.dones: dones, model.rates: rates}
            eloss, _ = sess.run([model.e_loss, model.e_opt], feed_dict)
            aloss, _ = sess.run([model.a_loss, model.a_opt], feed_dict)
            eloss_batch.append(eloss)
            aloss_batch.append(aloss)
            # End of episode/result/success rate available
            if done is True:
                # Rating the latest played episode
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                break

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        
        # Break episode/epoch loop
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of 500 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:17.0000 R:17.0000 rate:0.0340 aloss:-0.3665 eloss:4.1107 exploreP:0.9983
Episode:1 meanR:21.5000 R:26.0000 rate:0.0520 aloss:-0.2882 eloss:3.9948 exploreP:0.9958
Episode:2 meanR:19.3333 R:15.0000 rate:0.0300 aloss:-0.4877 eloss:4.2906 exploreP:0.9943
Episode:3 meanR:20.7500 R:25.0000 rate:0.0500 aloss:-0.2650 eloss:3.9523 exploreP:0.9918
Episode:4 meanR:20.2000 R:18.0000 rate:0.0360 aloss:-0.2235 eloss:3.8969 exploreP:0.9901
Episode:5 meanR:20.8333 R:24.0000 rate:0.0480 aloss:-0.2887 eloss:4.0132 exploreP:0.9877
Episode:6 meanR:20.5714 R:19.0000 rate:0.0380 aloss:-0.2152 eloss:3.8745 exploreP:0.9858
Episode:7 meanR:20.5000 R:20.0000 rate:0.0400 aloss:-0.1842 eloss:3.8401 exploreP:0.9839
Episode:8 meanR:23.8889 R:51.0000 rate:0.1020 aloss:-0.2077 eloss:3.8755 exploreP:0.9789
Episode:9 meanR:23.4000 R:19.0000 rate:0.0380 aloss:-0.1881 eloss:3.8501 exploreP:0.9771
Episode:10 meanR:24.2727 R:33.0000 rate:0.0660 aloss:-0.1627 eloss:3.7988 exploreP:0.9739
Episode:11 meanR:23.

Episode:92 meanR:20.6452 R:12.0000 rate:0.0240 aloss:0.8284 eloss:2.6117 exploreP:0.8271
Episode:93 meanR:20.8191 R:37.0000 rate:0.0740 aloss:0.8173 eloss:2.6251 exploreP:0.8240
Episode:94 meanR:20.7368 R:13.0000 rate:0.0260 aloss:0.8357 eloss:2.5810 exploreP:0.8230
Episode:95 meanR:20.6875 R:16.0000 rate:0.0320 aloss:0.8522 eloss:2.5574 exploreP:0.8217
Episode:96 meanR:20.6082 R:13.0000 rate:0.0260 aloss:0.8222 eloss:2.6608 exploreP:0.8206
Episode:97 meanR:20.6020 R:20.0000 rate:0.0400 aloss:0.8343 eloss:2.5980 exploreP:0.8190
Episode:98 meanR:20.6970 R:30.0000 rate:0.0600 aloss:0.8182 eloss:2.6448 exploreP:0.8166
Episode:99 meanR:20.6500 R:16.0000 rate:0.0320 aloss:0.8303 eloss:2.6050 exploreP:0.8153
Episode:100 meanR:20.6500 R:17.0000 rate:0.0340 aloss:0.8014 eloss:2.6812 exploreP:0.8139
Episode:101 meanR:20.5400 R:15.0000 rate:0.0300 aloss:0.8428 eloss:2.5687 exploreP:0.8127
Episode:102 meanR:20.5200 R:13.0000 rate:0.0260 aloss:0.8252 eloss:2.6390 exploreP:0.8117
Episode:103 meanR:

Episode:184 meanR:16.2400 R:13.0000 rate:0.0260 aloss:1.0450 eloss:2.3896 exploreP:0.7137
Episode:185 meanR:16.2900 R:18.0000 rate:0.0360 aloss:1.0556 eloss:2.3953 exploreP:0.7125
Episode:186 meanR:16.3600 R:17.0000 rate:0.0340 aloss:1.0732 eloss:2.3173 exploreP:0.7113
Episode:187 meanR:16.4000 R:16.0000 rate:0.0320 aloss:1.0680 eloss:2.3458 exploreP:0.7102
Episode:188 meanR:16.1000 R:14.0000 rate:0.0280 aloss:1.0838 eloss:2.2711 exploreP:0.7092
Episode:189 meanR:16.0500 R:11.0000 rate:0.0220 aloss:1.0998 eloss:2.3463 exploreP:0.7084
Episode:190 meanR:16.1400 R:20.0000 rate:0.0400 aloss:1.1044 eloss:2.3035 exploreP:0.7070
Episode:191 meanR:16.1200 R:11.0000 rate:0.0220 aloss:1.1143 eloss:2.2669 exploreP:0.7062
Episode:192 meanR:16.1600 R:16.0000 rate:0.0320 aloss:1.1079 eloss:2.2882 exploreP:0.7051
Episode:193 meanR:15.9200 R:13.0000 rate:0.0260 aloss:1.1376 eloss:2.2209 exploreP:0.7042
Episode:194 meanR:15.9800 R:19.0000 rate:0.0380 aloss:1.1419 eloss:2.2566 exploreP:0.7029
Episode:19

Episode:276 meanR:14.4000 R:17.0000 rate:0.0340 aloss:1.4700 eloss:1.8911 exploreP:0.6264
Episode:277 meanR:14.4400 R:17.0000 rate:0.0340 aloss:1.4845 eloss:1.9031 exploreP:0.6254
Episode:278 meanR:14.4000 R:10.0000 rate:0.0200 aloss:1.4781 eloss:1.8971 exploreP:0.6247
Episode:279 meanR:14.3200 R:10.0000 rate:0.0200 aloss:1.4890 eloss:1.9267 exploreP:0.6241
Episode:280 meanR:14.3300 R:20.0000 rate:0.0400 aloss:1.4880 eloss:1.8617 exploreP:0.6229
Episode:281 meanR:14.3300 R:13.0000 rate:0.0260 aloss:1.4430 eloss:1.9170 exploreP:0.6221
Episode:282 meanR:14.3700 R:18.0000 rate:0.0360 aloss:1.4915 eloss:1.9414 exploreP:0.6210
Episode:283 meanR:14.5500 R:29.0000 rate:0.0580 aloss:1.4848 eloss:1.9404 exploreP:0.6192
Episode:284 meanR:14.5800 R:16.0000 rate:0.0320 aloss:1.4428 eloss:1.9968 exploreP:0.6183
Episode:285 meanR:14.5500 R:15.0000 rate:0.0300 aloss:1.4847 eloss:1.9173 exploreP:0.6173
Episode:286 meanR:14.5000 R:12.0000 rate:0.0240 aloss:1.3810 eloss:1.9203 exploreP:0.6166
Episode:28

Episode:368 meanR:13.5600 R:9.0000 rate:0.0180 aloss:2.0017 eloss:1.4190 exploreP:0.5546
Episode:369 meanR:13.5800 R:14.0000 rate:0.0280 aloss:1.9655 eloss:1.4364 exploreP:0.5539
Episode:370 meanR:13.5400 R:10.0000 rate:0.0200 aloss:1.8257 eloss:1.6330 exploreP:0.5533
Episode:371 meanR:13.4800 R:9.0000 rate:0.0180 aloss:1.9817 eloss:1.5298 exploreP:0.5528
Episode:372 meanR:13.3300 R:10.0000 rate:0.0200 aloss:2.0117 eloss:1.4596 exploreP:0.5523
Episode:373 meanR:13.3700 R:16.0000 rate:0.0320 aloss:2.0193 eloss:1.4775 exploreP:0.5514
Episode:374 meanR:13.4600 R:21.0000 rate:0.0420 aloss:2.0186 eloss:1.5114 exploreP:0.5503
Episode:375 meanR:13.4700 R:12.0000 rate:0.0240 aloss:1.9829 eloss:1.4012 exploreP:0.5496
Episode:376 meanR:13.4800 R:18.0000 rate:0.0360 aloss:1.9735 eloss:1.5122 exploreP:0.5487
Episode:377 meanR:13.4300 R:12.0000 rate:0.0240 aloss:2.0219 eloss:1.4493 exploreP:0.5480
Episode:378 meanR:13.4400 R:11.0000 rate:0.0220 aloss:2.0139 eloss:1.4800 exploreP:0.5474
Episode:379 

...

Episode:2361 meanR:12.6100 R:10.0000 rate:0.0200 aloss:10.0032 eloss:-5.4154 exploreP:0.0615
Episode:2362 meanR:12.5900 R:8.0000 rate:0.0160 aloss:8.0821 eloss:-3.8317 exploreP:0.0615
Episode:2363 meanR:12.6000 R:11.0000 rate:0.0220 aloss:9.3160 eloss:-4.8237 exploreP:0.0614
Episode:2364 meanR:12.6200 R:12.0000 rate:0.0240 aloss:8.2123 eloss:-4.0162 exploreP:0.0614
Episode:2365 meanR:12.6400 R:10.0000 rate:0.0200 aloss:10.3503 eloss:-5.6184 exploreP:0.0613
Episode:2366 meanR:12.6400 R:9.0000 rate:0.0180 aloss:8.1280 eloss:-3.6892 exploreP:0.0613
Episode:2367 meanR:12.6700 R:12.0000 rate:0.0240 aloss:9.6338 eloss:-5.1049 exploreP:0.0612
Episode:2368 meanR:12.6400 R:9.0000 rate:0.0180 aloss:9.9358 eloss:-5.3582 exploreP:0.0612
Episode:2369 meanR:12.6300 R:10.0000 rate:0.0200 aloss:9.2381 eloss:-4.7380 exploreP:0.0611
Episode:2370 meanR:12.6100 R:10.0000 rate:0.0200 aloss:7.0296 eloss:-2.5615 exploreP:0.0610
Episode:2371 meanR:12.6400 R:13.0000 rate:0.0260 aloss:9.0663 eloss:-4.6562 explo

Episode:2451 meanR:9.8000 R:10.0000 rate:0.0200 aloss:9.2825 eloss:-4.8000 exploreP:0.0572
Episode:2452 meanR:9.8100 R:9.0000 rate:0.0180 aloss:9.7384 eloss:-5.5133 exploreP:0.0571
Episode:2453 meanR:9.8000 R:9.0000 rate:0.0180 aloss:9.7225 eloss:-5.2396 exploreP:0.0571
Episode:2454 meanR:9.8000 R:9.0000 rate:0.0180 aloss:9.6193 eloss:-5.1643 exploreP:0.0570
Episode:2455 meanR:9.8000 R:10.0000 rate:0.0200 aloss:9.7641 eloss:-5.3761 exploreP:0.0570
Episode:2456 meanR:9.7900 R:9.0000 rate:0.0180 aloss:10.0845 eloss:-5.7706 exploreP:0.0570
Episode:2457 meanR:9.7700 R:8.0000 rate:0.0160 aloss:9.2997 eloss:-4.8694 exploreP:0.0569
Episode:2458 meanR:9.7600 R:9.0000 rate:0.0180 aloss:9.7684 eloss:-5.2560 exploreP:0.0569
Episode:2459 meanR:9.7500 R:9.0000 rate:0.0180 aloss:8.4187 eloss:-4.2917 exploreP:0.0568
Episode:2460 meanR:9.7200 R:10.0000 rate:0.0200 aloss:10.2231 eloss:-5.8462 exploreP:0.0568
Episode:2461 meanR:9.7200 R:10.0000 rate:0.0200 aloss:8.3980 eloss:-4.1707 exploreP:0.0567
Epis

Episode:2542 meanR:9.6600 R:11.0000 rate:0.0220 aloss:7.4328 eloss:-2.8934 exploreP:0.0532
Episode:2543 meanR:9.6400 R:9.0000 rate:0.0180 aloss:7.9867 eloss:-3.6140 exploreP:0.0532
Episode:2544 meanR:9.6500 R:10.0000 rate:0.0200 aloss:10.0291 eloss:-5.5603 exploreP:0.0531
Episode:2545 meanR:9.6500 R:10.0000 rate:0.0200 aloss:8.5797 eloss:-4.1390 exploreP:0.0531
Episode:2546 meanR:9.6300 R:9.0000 rate:0.0180 aloss:10.3655 eloss:-5.8054 exploreP:0.0531
Episode:2547 meanR:9.6400 R:10.0000 rate:0.0200 aloss:10.3247 eloss:-5.6746 exploreP:0.0530
Episode:2548 meanR:9.6400 R:10.0000 rate:0.0200 aloss:9.9546 eloss:-5.3745 exploreP:0.0530
Episode:2549 meanR:9.6300 R:9.0000 rate:0.0180 aloss:9.4667 eloss:-5.2185 exploreP:0.0529
Episode:2550 meanR:9.6200 R:10.0000 rate:0.0200 aloss:9.7646 eloss:-5.3255 exploreP:0.0529
Episode:2551 meanR:9.6200 R:10.0000 rate:0.0200 aloss:9.8810 eloss:-5.5868 exploreP:0.0528
Episode:2552 meanR:9.6100 R:8.0000 rate:0.0160 aloss:8.9036 eloss:-4.8888 exploreP:0.0528


Episode:2633 meanR:9.4000 R:10.0000 rate:0.0200 aloss:8.0257 eloss:-3.8168 exploreP:0.0497
Episode:2634 meanR:9.3900 R:9.0000 rate:0.0180 aloss:9.8539 eloss:-5.2555 exploreP:0.0497
Episode:2635 meanR:9.4100 R:11.0000 rate:0.0220 aloss:10.3029 eloss:-5.6813 exploreP:0.0496
Episode:2636 meanR:9.4100 R:9.0000 rate:0.0180 aloss:9.1623 eloss:-4.7279 exploreP:0.0496
Episode:2637 meanR:9.4000 R:9.0000 rate:0.0180 aloss:7.4934 eloss:-3.4014 exploreP:0.0495
Episode:2638 meanR:9.4000 R:9.0000 rate:0.0180 aloss:8.4252 eloss:-4.0989 exploreP:0.0495
Episode:2639 meanR:9.4000 R:10.0000 rate:0.0200 aloss:9.9032 eloss:-5.4253 exploreP:0.0495
Episode:2640 meanR:9.4000 R:10.0000 rate:0.0200 aloss:7.4126 eloss:-3.2578 exploreP:0.0494
Episode:2641 meanR:9.4100 R:9.0000 rate:0.0180 aloss:9.4330 eloss:-5.2462 exploreP:0.0494
Episode:2642 meanR:9.3900 R:9.0000 rate:0.0180 aloss:8.6904 eloss:-4.5520 exploreP:0.0494
Episode:2643 meanR:9.3800 R:8.0000 rate:0.0160 aloss:8.8121 eloss:-4.5209 exploreP:0.0493
Episo

Episode:2724 meanR:9.4600 R:9.0000 rate:0.0180 aloss:9.3317 eloss:-4.8447 exploreP:0.0464
Episode:2725 meanR:9.4700 R:10.0000 rate:0.0200 aloss:8.2516 eloss:-4.0872 exploreP:0.0464
Episode:2726 meanR:9.4800 R:9.0000 rate:0.0180 aloss:1.6753 eloss:2.6943 exploreP:0.0463
Episode:2727 meanR:9.4700 R:8.0000 rate:0.0160 aloss:10.6765 eloss:-6.0170 exploreP:0.0463
Episode:2728 meanR:9.4700 R:11.0000 rate:0.0220 aloss:9.6819 eloss:-5.1449 exploreP:0.0463
Episode:2729 meanR:9.4800 R:9.0000 rate:0.0180 aloss:8.4226 eloss:-4.1581 exploreP:0.0462
Episode:2730 meanR:9.4600 R:8.0000 rate:0.0160 aloss:9.0574 eloss:-4.9458 exploreP:0.0462
Episode:2731 meanR:9.4700 R:10.0000 rate:0.0200 aloss:8.4390 eloss:-3.8338 exploreP:0.0462
Episode:2732 meanR:9.4600 R:9.0000 rate:0.0180 aloss:9.3839 eloss:-5.1194 exploreP:0.0461
Episode:2733 meanR:9.4400 R:8.0000 rate:0.0160 aloss:2.9711 eloss:1.6099 exploreP:0.0461
Episode:2734 meanR:9.4500 R:10.0000 rate:0.0200 aloss:8.5973 eloss:-4.5538 exploreP:0.0461
Episode

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
